In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up the Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the appropriate WebDriver (e.g., Firefox, Edge)
driver.maximize_window()

# Open the target webpage
url = "https://www.nasdaq.com/market-activity/stocks/screener"  # Replace with the actual URL
driver.get(url)

# Wait for the page to load and the "Download CSV" button to appear
try:
    wait = WebDriverWait(driver, 15)
    download_button = wait.until(
        EC.element_to_be_clickable((By.CLASS_NAME, "jupiter22-c-table__download-csv"))
    )

    # Click the "Download CSV" button
    download_button.click()
    print("Download button clicked. File should start downloading.")

    # Optional: Wait for a few seconds to ensure download starts
    time.sleep(5)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the browser
    driver.quit()

Download button clicked. File should start downloading.


In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# Read symbols from CSV file
input_file = r"C:\Users\dynam\Downloads\nasdaq_screener_1737451177628.csv"  # Replace with your file path
df = pd.read_csv(input_file)
symbols = df['Symbol'].tolist()

# Output data
all_data = []

# Selenium WebDriver setup
driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
wait = WebDriverWait(driver, 15)

def scrape_ticker_data_from_tab(ticker):
    try:
        # Switch to tab
        driver.switch_to.window(driver.window_handles[-1])
        
        url = f"https://www.nasdaq.com/market-activity/stocks/{ticker}/sec-filings?page=1&rows_per_page=14"
        driver.get(url)
        time.sleep(2)  # Allow page to load

        # Handle cookie policy or pop-ups
        try:
            policy_popup = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
            )
            policy_popup.click()
        except Exception:
            pass  # No popup found

        # Extract table rows
        while True:
            try:
                rows = wait.until(EC.presence_of_all_elements_located(
                    (By.XPATH, "//tbody[@class='jupiter22-c-sec-filings-table__body']//tr")))
                for row in rows:
                    try:
                        # Extract table columns
                        company_name = row.find_element(By.XPATH, "./td[1]").text
                        form_type = row.find_element(By.XPATH, "./td[2]").text
                        filing_date = row.find_element(By.XPATH, "./td[3]").text
                        filing_period = row.find_element(By.XPATH, "./td[4]").text
                        filing_link = row.find_element(By.XPATH, "./td[5]//a[text()='DOC']").get_attribute('href')

                        # Append data
                        all_data.append({
                            "Ticker": ticker,
                            "Company Name": company_name,
                            "Form Type": form_type,
                            "Filing Date": filing_date,
                            "Filing Period": filing_period,
                            "Filing Link": filing_link
                        })

                    except Exception as e:
                        print(f"Error extracting row data for ticker {ticker}: {e}")
                
                # Check if there is a next page
                try:
                    next_button = driver.find_element(By.XPATH, "//button[@class='pagination__next']")
                    if next_button.is_enabled():
                        next_button.click()
                        time.sleep(random.randint(2, 5))  # Random delay
                    else:
                        break
                except Exception:
                    break
            except Exception:
                break
    finally:
        print(f"Finished scraping for ticker: {ticker}")

# Batch processing
batch_size = 10  # Number of tabs per batch

try:
    for i in range(0, len(symbols), batch_size):
        batch = symbols[i:i + batch_size]
        
        # Open tabs for each ticker in the batch
        for ticker in batch:
            driver.execute_script("window.open('');")
            scrape_ticker_data_from_tab(ticker)
        
        # Close all tabs except the first one
        while len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        print(f"Processed batch {i // batch_size + 1} of {len(symbols) // batch_size + 1}")
finally:
    # Quit driver
    driver.quit()

# Save to CSV
output_file = "nasdaq_sec_filings.csv"
pd.DataFrame(all_data).to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

Finished scraping for ticker: A
Finished scraping for ticker: AA
Finished scraping for ticker: AACG
Finished scraping for ticker: AACT
Finished scraping for ticker: AADI
Finished scraping for ticker: AAL
Finished scraping for ticker: AAM
Finished scraping for ticker: AAME
Finished scraping for ticker: AAMI
Finished scraping for ticker: AAOI
Processed batch 1 of 697
Finished scraping for ticker: AAON
Finished scraping for ticker: AAP
Finished scraping for ticker: AAPL
Finished scraping for ticker: AAT
Finished scraping for ticker: AB
Finished scraping for ticker: ABAT
Finished scraping for ticker: ABBV
Finished scraping for ticker: ABCB
Finished scraping for ticker: ABCL
Finished scraping for ticker: ABEO
Processed batch 2 of 697
Finished scraping for ticker: ABEV
Finished scraping for ticker: ABG
Finished scraping for ticker: ABL
Finished scraping for ticker: ABLLL
Finished scraping for ticker: ABLLW
Finished scraping for ticker: ABLV
Finished scraping for ticker: ABLVW
Finished scrapi

Processed batch 22 of 697
Finished scraping for ticker: AIRS
Finished scraping for ticker: AIRT
Finished scraping for ticker: AIRTP
Finished scraping for ticker: AISP
Finished scraping for ticker: AISPW
Finished scraping for ticker: AIT
Finished scraping for ticker: AITR
Finished scraping for ticker: AITRR
Finished scraping for ticker: AIV
Finished scraping for ticker: AIXI
Processed batch 23 of 697
Finished scraping for ticker: AIZ
Finished scraping for ticker: AIZN
Finished scraping for ticker: AJG
Finished scraping for ticker: AKA
Finished scraping for ticker: AKAM
Finished scraping for ticker: AKAN
Finished scraping for ticker: AKBA
Finished scraping for ticker: AKO/A
Finished scraping for ticker: AKO/B
Finished scraping for ticker: AKR
Processed batch 24 of 697
Finished scraping for ticker: AKRO
Finished scraping for ticker: AKTX
Finished scraping for ticker: AKYA
Finished scraping for ticker: AL
Finished scraping for ticker: ALAB
Finished scraping for ticker: ALAR
Finished scrapi

Finished scraping for ticker: AMT
Finished scraping for ticker: AMTB
Finished scraping for ticker: AMTD
Finished scraping for ticker: AMTM
Finished scraping for ticker: AMTX
Finished scraping for ticker: AMWD
Finished scraping for ticker: AMWL
Finished scraping for ticker: AMX
Finished scraping for ticker: AMZN
Finished scraping for ticker: AN
Processed batch 36 of 697
Finished scraping for ticker: ANAB
Finished scraping for ticker: ANDE
Finished scraping for ticker: ANEB
Finished scraping for ticker: ANET
Finished scraping for ticker: ANF
Finished scraping for ticker: ANG^A
Finished scraping for ticker: ANG^B
Finished scraping for ticker: ANG^D
Finished scraping for ticker: ANGH
Finished scraping for ticker: ANGHW
Processed batch 37 of 697
Finished scraping for ticker: ANGI
Finished scraping for ticker: ANGO
Finished scraping for ticker: ANIK
Finished scraping for ticker: ANIP
Finished scraping for ticker: ANIX
Finished scraping for ticker: ANL
Finished scraping for ticker: ANNA
Finis

Finished scraping for ticker: ATOS
Processed batch 57 of 697
Finished scraping for ticker: ATPC
Finished scraping for ticker: ATR
Finished scraping for ticker: ATRA
Finished scraping for ticker: ATRC
Finished scraping for ticker: ATRO
Finished scraping for ticker: ATS
Finished scraping for ticker: ATSG
Finished scraping for ticker: ATUS
Finished scraping for ticker: ATXG
Finished scraping for ticker: ATXI
Processed batch 58 of 697
Finished scraping for ticker: ATXS
Finished scraping for ticker: ATYR
Finished scraping for ticker: AU
Finished scraping for ticker: AUB
Finished scraping for ticker: AUB^A
Finished scraping for ticker: AUBN
Finished scraping for ticker: AUDC
Finished scraping for ticker: AUID
Finished scraping for ticker: AUNA
Finished scraping for ticker: AUPH
Processed batch 59 of 697
Finished scraping for ticker: AUR
Finished scraping for ticker: AURA
Finished scraping for ticker: AUROW
Finished scraping for ticker: AUST
Finished scraping for ticker: AUTL
Finished scrapin

Finished scraping for ticker: BEN
Finished scraping for ticker: BENF
Finished scraping for ticker: BENFW
Finished scraping for ticker: BEP
Finished scraping for ticker: BEP^A
Processed batch 78 of 697
Finished scraping for ticker: BEPC
Finished scraping for ticker: BEPH
Finished scraping for ticker: BEPI
Finished scraping for ticker: BEPJ
Finished scraping for ticker: BERY
Finished scraping for ticker: BEST
Finished scraping for ticker: BETR
Finished scraping for ticker: BETRW
Finished scraping for ticker: BF/A
Finished scraping for ticker: BF/B
Processed batch 79 of 697
Finished scraping for ticker: BFAM
Finished scraping for ticker: BFC
Finished scraping for ticker: BFH
Finished scraping for ticker: BFIN
Finished scraping for ticker: BFK
Finished scraping for ticker: BFLY
Finished scraping for ticker: BFRG
Finished scraping for ticker: BFRGW
Finished scraping for ticker: BFRI
Finished scraping for ticker: BFRIW
Processed batch 80 of 697
Finished scraping for ticker: BFS
Finished scra

...

Finished scraping for ticker: T
Finished scraping for ticker: T^A
Finished scraping for ticker: T^C
Processed batch 604 of 697
Finished scraping for ticker: TAC
Finished scraping for ticker: TACT
Finished scraping for ticker: TAIT
Finished scraping for ticker: TAK
Finished scraping for ticker: TAL
Finished scraping for ticker: TALK
Finished scraping for ticker: TALKW
Finished scraping for ticker: TALO
Finished scraping for ticker: TANH
Finished scraping for ticker: TAOP
Processed batch 605 of 697
Finished scraping for ticker: TAP
Finished scraping for ticker: TARA
Finished scraping for ticker: TARS
Finished scraping for ticker: TASK
Finished scraping for ticker: TATT
Finished scraping for ticker: TAVI
Finished scraping for ticker: TAVIR
Finished scraping for ticker: TAVIU
Finished scraping for ticker: TAYD
Finished scraping for ticker: TBB
Processed batch 606 of 697
Finished scraping for ticker: TBBB
Finished scraping for ticker: TBBK
Finished scraping for ticker: TBCH
Finished scrapin

Finished scraping for ticker: THTX
Finished scraping for ticker: THW
Finished scraping for ticker: TIGO
Finished scraping for ticker: TIGR
Finished scraping for ticker: TIL
Finished scraping for ticker: TILE
Finished scraping for ticker: TIMB
Finished scraping for ticker: TIPT
Finished scraping for ticker: TIRX
Processed batch 617 of 697
Finished scraping for ticker: TISI
Finished scraping for ticker: TITN
Finished scraping for ticker: TIVC
Finished scraping for ticker: TIXT
Finished scraping for ticker: TJX
Finished scraping for ticker: TK
Finished scraping for ticker: TKC
Finished scraping for ticker: TKLF
Finished scraping for ticker: TKNO
Finished scraping for ticker: TKO
Processed batch 618 of 697
Finished scraping for ticker: TKR
Finished scraping for ticker: TLF
Finished scraping for ticker: TLK
Finished scraping for ticker: TLN
Finished scraping for ticker: TLPH
Finished scraping for ticker: TLRY
Finished scraping for ticker: TLS
Finished scraping for ticker: TLSA
Finished scra

Finished scraping for ticker: UEIC
Processed batch 638 of 697
Finished scraping for ticker: UFCS
Finished scraping for ticker: UFG
Finished scraping for ticker: UFI
Finished scraping for ticker: UFPI
Finished scraping for ticker: UFPT
Finished scraping for ticker: UG
Finished scraping for ticker: UGI
Finished scraping for ticker: UGP
Finished scraping for ticker: UGRO
Finished scraping for ticker: UHAL
Processed batch 639 of 697
Finished scraping for ticker: UHG
Finished scraping for ticker: UHGWW
Finished scraping for ticker: UHS
Finished scraping for ticker: UHT
Finished scraping for ticker: UI
Finished scraping for ticker: UIS
Finished scraping for ticker: UK
Finished scraping for ticker: UKOMW
Finished scraping for ticker: UL
Finished scraping for ticker: ULBI
Processed batch 640 of 697
Finished scraping for ticker: ULCC
Finished scraping for ticker: ULH
Finished scraping for ticker: ULS
Finished scraping for ticker: ULTA
Finished scraping for ticker: ULY
Finished scraping for tick

Finished scraping for ticker: VRE
Finished scraping for ticker: VREX
Processed batch 660 of 697
Finished scraping for ticker: VRME
Finished scraping for ticker: VRMEW
Finished scraping for ticker: VRN
Finished scraping for ticker: VRNA
Finished scraping for ticker: VRNS
Finished scraping for ticker: VRNT
Finished scraping for ticker: VRPX
Finished scraping for ticker: VRRM
Finished scraping for ticker: VRSK
Finished scraping for ticker: VRSN
Processed batch 661 of 697
Finished scraping for ticker: VRT
Finished scraping for ticker: VRTS
Finished scraping for ticker: VRTX
Finished scraping for ticker: VS
Finished scraping for ticker: VSAT
Finished scraping for ticker: VSCO
Finished scraping for ticker: VSEC
Finished scraping for ticker: VSEE
Finished scraping for ticker: VSEEW
Finished scraping for ticker: VSH
Processed batch 662 of 697
Finished scraping for ticker: VSME
Finished scraping for ticker: VSSYW
Finished scraping for ticker: VST
Finished scraping for ticker: VSTA
Finished scra

Finished scraping for ticker: WY
Finished scraping for ticker: WYHG
Processed batch 682 of 697
Finished scraping for ticker: WYNN
Finished scraping for ticker: WYY
Finished scraping for ticker: X
Finished scraping for ticker: XAIR
Finished scraping for ticker: XBIO
Finished scraping for ticker: XBIT
Finished scraping for ticker: XBP
Finished scraping for ticker: XBPEW
Finished scraping for ticker: XCH
Finished scraping for ticker: XCUR
Processed batch 683 of 697
Finished scraping for ticker: XEL
Finished scraping for ticker: XELB
Finished scraping for ticker: XENE
Finished scraping for ticker: XERS
Finished scraping for ticker: XFLT
Finished scraping for ticker: XFLT^A
Finished scraping for ticker: XFOR
Finished scraping for ticker: XGN
Finished scraping for ticker: XHG
Finished scraping for ticker: XHR
Processed batch 684 of 697
Finished scraping for ticker: XIN
Finished scraping for ticker: XLO
Finished scraping for ticker: XMTR
Finished scraping for ticker: XNCR
Finished scraping fo

In [ ]:
import pandas as pd

# Define the mappings
mappings = {
    # Annual Reports
    "10-K": "Annual Reports",
    "10-K/A": "Annual Reports",
    "20-F": "Annual Reports",
    "40-F": "Annual Reports",
    "10-KT": "Annual Reports",

    # Quarterly Reports
    "10-Q": "Quarterly Reports",
    "10-Q/A": "Quarterly Reports",
    "NT 10-Q": "Quarterly Reports",
    "NT 10-Q/A": "Quarterly Reports",

    # Proxies and Info Statements
    "DEF 14A": "Proxies and Info Statements",
    "DEFA14A": "Proxies and Info Statements",
    "PRE 14A": "Proxies and Info Statements",
    "PRER14A": "Proxies and Info Statements",
    "DEFM14A": "Proxies and Info Statements",
    "PREC14A": "Proxies and Info Statements",
    "DEF 14C": "Proxies and Info Statements",
    "PRE 14C": "Proxies and Info Statements",
    "435": "Proxies and Info Statements",

    # Insider Transactions
    "3": "Insider Transactions",
    "4": "Insider Transactions",
    "4/A": "Insider Transactions",
    "5": "Insider Transactions",
    "SC 13G/A": "Insider Transactions",

    # 8-K Related
    "8-K": "8-K Related",
    "8-K/A": "8-K Related",

    # Registration Statements
    "S-1": "Registration Statements",
    "S-3": "Registration Statements",
    "S-8": "Registration Statements",
    "424B1": "Registration Statements",
    "424B2": "Registration Statements",
    "424B3": "Registration Statements",
    "424B4": "Registration Statements",
    "424B5": "Registration Statements",
    "424B7": "Registration Statements",
    "424B8": "Registration Statements",
    "F-1": "Registration Statements",
    "F-3": "Registration Statements",
    "F-4": "Registration Statements",
    "S-11": "Registration Statements",
    "FWP":"Registration Statements",
    "S-3ASR": "Registration Statements",
    "8-A12B/A": "Registration Statements",
    "EFFECT" : "Registration Statements",
    "POS AM" : "Registration Statements",
    "S-3" : "Registration Statements",
    "S-4/A": "Registration Statements",
    "S-1/A": "Registration Statements",

    # Comment Letters
    "CORRESP": "Comment Letters",
    "UPLOAD": "Comment Letters",

    # Other Examples
    #"SD": "None",
    #"144": "None",
    #"13F-NT": "None",
    #"13F-HR": "None",
    #"ABS-15G": "None",
    #"SC 13G/A": "None",
    #"SC 13G": "None",
    #"SCHEDULE 13G": "None",
    #"6-K": "None",
    #"SC 13D/A":"None",
    #"SC 13D": "None",
    #"SCHEDULE 13D": "None",
    #"25": "None",
    #"25-NSE": "None",
    #"25-NSE": "None",
    #"RW": "None",
}

# Example of mapping to be applied in a dataset
def apply_mapping(file_path, output_file_path):
    # Load the data
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .csv or Excel file.")

    # Apply the mapping to the 'Form Type' column and create a new 'Form Group' column
    if 'Form Type' in df.columns:
        df.insert(df.columns.get_loc('Form Type') + 1, 'Form Group', df['Form Type'].map(mappings))
    else:
        raise KeyError("The input file does not contain a 'Form Type' column.")

    # Save the updated DataFrame to a new file
    if output_file_path.endswith('.csv'):
        df.to_csv(output_file_path, index=False)
    elif output_file_path.endswith(('.xls', '.xlsx')):
        df.to_excel(output_file_path, index=False)
    else:
        raise ValueError("Unsupported output file format. Please specify a .csv or Excel file.")

# Example usage
apply_mapping(r'D:\\coffee_disease_project\\nasdaq_sec_filings.csv', 'final_nasdaq_sec_filings.csv')

In [2]:
import pandas as pd

# Load the CSV files
file2 = r"C:\Users\dynam\Desktop\USA\nasdaq_screener_1737686840397.csv"  # New file
file1 = r"C:\Users\dynam\Downloads\nasdaq_screener_1737451177628.csv"  # previous file

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Find unique symbols in file2 but not in file1
symbols_file1 = set(df1['Symbol'])
symbols_file2 = set(df2['Symbol'])

unique_to_file2 = symbols_file2 - symbols_file1

# Save unique symbols to a new CSV
unique_symbols_file = "unique_symbols.csv"
pd.DataFrame(list(unique_to_file2), columns=["Symbol"]).to_csv(unique_symbols_file, index=False)

print(f"Unique symbols saved to {unique_symbols_file}")

Unique symbols saved to unique_symbols.csv


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# Read unique symbols from the CSV file
input_file = "unique_symbols.csv"  # File with unique symbols from Step 1
df = pd.read_csv(input_file)
symbols = df['Symbol'].tolist()

# Output data
all_data = []

# Selenium WebDriver setup
driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
wait = WebDriverWait(driver, 15)

def scrape_ticker_data(ticker):
    url = f"https://www.nasdaq.com/market-activity/stocks/{ticker}/sec-filings?page=1&rows_per_page=14"
    driver.get(url)
    time.sleep(2)  # Allow page to load

    # Handle cookie policy or pop-ups
    try:
        policy_popup = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))  # Adjust selector as needed
        )
        policy_popup.click()
        print("Closed cookie policy popup.")
    except Exception:
        print("No cookie policy popup found.")

    while True:
        try:
            # Extract table rows
            rows = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//tbody[@class='jupiter22-c-sec-filings-table__body']//tr")))
            for row in rows:
                try:
                    # Extract table columns
                    company_name = row.find_element(By.XPATH, "./td[1]").text
                    form_type = row.find_element(By.XPATH, "./td[2]").text
                    filing_date = row.find_element(By.XPATH, "./td[3]").text
                    filing_period = row.find_element(By.XPATH, "./td[4]").text
                    filing_link = row.find_element(By.XPATH, "./td[5]//a[text()='DOC']").get_attribute('href')
                    
                    # Append data
                    all_data.append({
                        "Ticker": ticker,
                        "Company Name": company_name,
                        "Form Type": form_type,
                        "Filing Date": filing_date,
                        "Filing Period": filing_period,
                        "Filing Link": filing_link
                    })

                    # Print row data for debugging
                    print(f"Extracted row for {ticker}: {company_name}, {form_type}, {filing_date}, {filing_period}, {filing_link}")

                except Exception as e:
                    print(f"Error extracting row data for ticker {ticker}: {e}")
            
            # Check if there is a next page
            try:
                next_button = driver.find_element(By.XPATH, "//button[@class='pagination__next']")
                if next_button.is_enabled():
                    next_button.click()
                    print(f"Navigated to next page for ticker {ticker}.")
                    time.sleep(random.randint(2, 5))  # Random delay between page loads
                else:
                    break
            except Exception as e:
                print(f"No more pages or error navigating: {e}")
                break

        except Exception as e:
            print(f"Error extracting table data for ticker {ticker}: {e}")
            break

    # Print summary for this ticker
    print(f"Finished scraping for ticker: {ticker}. Total records: {len([d for d in all_data if d['Ticker'] == ticker])}\n")

try:
    for symbol in symbols:
        print(f"Scraping data for ticker: {symbol}")
        scrape_ticker_data(symbol)
        time.sleep(random.randint(3, 6))  # Random delay between tickers to mimic human behavior
finally:
    # Quit driver
    driver.quit()

# Save to CSV
# Save to CSV
output_file = "nasdaq_sec_filings.csv"

# Check if the file already exists
try:
    existing_data = pd.read_csv(output_file)
    # Append new data to existing data
    updated_data = pd.concat([existing_data, pd.DataFrame(all_data)], ignore_index=True)
    updated_data.to_csv(output_file, index=False)
    print(f"Data appended to {output_file}")
except FileNotFoundError:
    # If the file doesn't exist, create it with the new data
    pd.DataFrame(all_data).to_csv(output_file, index=False)
    print(f"Data saved to {output_file} (new file created)")

Scraping data for ticker: DXST
Closed cookie policy popup.
Error extracting table data for ticker DXST: Message: 

Finished scraping for ticker: DXST. Total records: 0

Scraping data for ticker: CLRCU
No cookie policy popup found.
Extracted row for CLRCU: ClimateRock Unit, SC 13G, 11/14/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318724451&type=DOC&symbol=CLRCU&cdn=9a084ca3bc6907494e628cf34e852c04&companyName=ClimateRock+Unit&formType=SC+13G&formDescription=Statement+of+acquisition+of+beneficial+ownership+by+individuals&dateFiled=2024-11-14
Extracted row for CLRCU: ClimateRock Unit, 10-Q, 11/14/2024, 09/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318721199&type=DOC&symbol=CLRCU&cdn=cc3f07683faee226e9d9b8e97b22dfef&companyName=ClimateRock+Unit&formType=10-Q&formDescription=General+form+for+quarterly+reports+under+Section+13+or+15%28d%29&dateFiled=2024-11-14
Extracted row for CLRCU: ClimateRock Unit, SC 13G/A, 11/14/2024,

Scraping data for ticker: HCVIW
No cookie policy popup found.
Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, SCHEDULE 13G/A, 01/10/2025, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318837665&type=DOC&symbol=HCVIW&cdn=74d252cffefb9c2270bc978c4cf387e4&companyName=Hennessy+Capital+Investment+Corp.+VI+Warrant&formType=SCHEDULE+13G%2FA&formDescription=%5BAmend%5D+Statement+of+acquisition+of+beneficial+ownership+by+individuals&dateFiled=2025-01-10
Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, 8-K, 12/20/2024, 12/19/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318801362&type=DOC&symbol=HCVIW&cdn=c4a124bcea6e5f8c8183e5397caaed5a&companyName=Hennessy+Capital+Investment+Corp.+VI+Warrant&formType=8-K&formDescription=Current+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2024-12-20
Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, 425, 12/09/2024, , https://app.quote

Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, 8-K, 09/26/2024, 09/26/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318612507&type=DOC&symbol=HCVIW&cdn=cdd0a122014cd1fcd8bb6f399cc73997&companyName=Hennessy+Capital+Investment+Corp.+VI+Warrant&formType=8-K&formDescription=Current+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2024-09-26
Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, 425, 09/24/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318605457&type=DOC&symbol=HCVIW&cdn=e2aaa284eb4cc423a6e09c11fbd567c4&companyName=Hennessy+Capital+Investment+Corp.+VI+Warrant&formType=425&formDescription=Prospectuses+and+communications%2C+business+combinations&dateFiled=2024-09-24
Extracted row for HCVIW: Hennessy Capital Investment Corp. VI Warrant, 8-K, 09/24/2024, 09/24/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318605448&type=DOC&symbol=HCVIW&cdn=185b248

Extracted row for ATMCW: AlphaTime Acquisition Corp Warrant, SC 13G, 11/14/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318722515&type=DOC&symbol=ATMCW&cdn=efd7c7d69b174ac17cf5443f67b68392&companyName=AlphaTime+Acquisition+Corp+Warrant&formType=SC+13G&formDescription=Statement+of+acquisition+of+beneficial+ownership+by+individuals&dateFiled=2024-11-14
Extracted row for ATMCW: AlphaTime Acquisition Corp Warrant, NT 10-Q, 11/14/2024, 09/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318721878&type=DOC&symbol=ATMCW&cdn=a2c9794aed54054da1770a06d5d2e0c9&companyName=AlphaTime+Acquisition+Corp+Warrant&formType=NT+10-Q&formDescription=Notification+of+inability+to+timely+file+Form+10-Q+or+10-QSB&dateFiled=2024-11-14
Extracted row for ATMCW: AlphaTime Acquisition Corp Warrant, SC 13G, 11/13/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318713732&type=DOC&symbol=ATMCW&cdn=70baaac4079d22c91d77352e049975c5&

Extracted row for FGIWW: FGI Industries Ltd. warrant, 8-K, 09/11/2024, 09/06/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318586748&type=DOC&symbol=FGIWW&cdn=12ffb054026b48e56271fec8e43eadcf&companyName=FGI+Industries+Ltd.+warrant&formType=8-K&formDescription=Current+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2024-09-11
Extracted row for FGIWW: FGI Industries Ltd. warrant, S-8, 08/22/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318542435&type=DOC&symbol=FGIWW&cdn=1985a590821327ab6f37958499c85cc9&companyName=FGI+Industries+Ltd.+warrant&formType=S-8&formDescription=Registration+statement+under+Securities+Act+of+1933+to+be+offered+to+employees+pursuant+to+certain+plans&dateFiled=2024-08-22
Extracted row for FGIWW: FGI Industries Ltd. warrant, 10-Q, 08/13/2024, 06/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318518496&type=DOC&symbol=FGIWW&cdn=0ed59d2b4252fa75677006b125a3fdb8&companyNam

...

Extracted row for CLNNW: Clene Inc. Warrant, 4, 10/03/2024, 10/02/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318629481&type=DOC&symbol=CLNNW&cdn=3af2da1f97c83592b1621aeb630b77c2&companyName=Clene+Inc.+Warrant&formType=4&formDescription=Statement+of+changes+in+beneficial+ownership+of+securities&dateFiled=2024-10-03
Extracted row for CLNNW: Clene Inc. Warrant, 4, 10/03/2024, 10/02/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318629498&type=DOC&symbol=CLNNW&cdn=302497d96b75d2778682a30b64a10acd&companyName=Clene+Inc.+Warrant&formType=4&formDescription=Statement+of+changes+in+beneficial+ownership+of+securities&dateFiled=2024-10-03
Navigated to next page for ticker CLNNW.
Extracted row for CLNNW: Clene Inc. Warrant, 4, 10/03/2024, 10/02/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318629383&type=DOC&symbol=CLNNW&cdn=dfba1cd1f6efd4b5bc20d5fde027f864&companyName=Clene+Inc.+Warrant&formType=4&formDescrip

Scraping data for ticker: NETDU
No cookie policy popup found.
Extracted row for NETDU: Nabors Energy Transition Corp. II Unit, SC 13G/A, 11/15/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318725243&type=DOC&symbol=NETDU&cdn=8e4bb71cbac947409d2cd382121890e3&companyName=Nabors+Energy+Transition+Corp.+II+Unit&formType=SC+13G%2FA&formDescription=%5BAmend%5D+Statement+of+acquisition+of+beneficial+ownership+by+individuals&dateFiled=2024-11-15
Extracted row for NETDU: Nabors Energy Transition Corp. II Unit, SC 13G, 11/14/2024, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318720751&type=DOC&symbol=NETDU&cdn=d5a87533075eb99cc798528a586c85ea&companyName=Nabors+Energy+Transition+Corp.+II+Unit&formType=SC+13G&formDescription=Statement+of+acquisition+of+beneficial+ownership+by+individuals&dateFiled=2024-11-14
Extracted row for NETDU: Nabors Energy Transition Corp. II Unit, SC 13G, 11/13/2024, , https://app.quotemedia.com/data/downloadFiling?

Navigated to next page for ticker PTIXW.
Extracted row for PTIXW: Protagenic Therapeutics Inc. Warrant, 10-Q, 11/14/2024, 09/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318722235&type=DOC&symbol=PTIXW&cdn=08bb7ae32e5cd68dd655a8c1dd617593&companyName=Protagenic+Therapeutics+Inc.+Warrant&formType=10-Q&formDescription=General+form+for+quarterly+reports+under+Section+13+or+15%28d%29&dateFiled=2024-11-14
Extracted row for PTIXW: Protagenic Therapeutics Inc. Warrant, 8-K, 11/05/2024, 11/04/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318688488&type=DOC&symbol=PTIXW&cdn=160868d25977c79dc7f9eb7dbad0b552&companyName=Protagenic+Therapeutics+Inc.+Warrant&formType=8-K&formDescription=Current+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2024-11-05
Extracted row for PTIXW: Protagenic Therapeutics Inc. Warrant, 8-K, 10/30/2024, 10/29/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318673142&type=DOC&sym

Extracted row for SVIIU: Spring Valley Acquisition Corp. II Units, PRE 14A, 09/30/2024, 09/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318620188&type=DOC&symbol=SVIIU&cdn=1894c270601fbcd41fff4da70fc373c2&companyName=Spring+Valley+Acquisition+Corp.+II+Units&formType=PRE+14A&formDescription=Preliminary+proxy+statement+providing+notification+matters+to+be+brought+to+a+vote&dateFiled=2024-09-30
Extracted row for SVIIU: Spring Valley Acquisition Corp. II Units, 10-Q, 08/08/2024, 06/30/2024, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318507143&type=DOC&symbol=SVIIU&cdn=401758ee8e3b97f7768ddea07eea77e8&companyName=Spring+Valley+Acquisition+Corp.+II+Units&formType=10-Q&formDescription=General+form+for+quarterly+reports+under+Section+13+or+15%28d%29&dateFiled=2024-08-08
Finished scraping for ticker: SVIIU. Total records: 20

Scraping data for ticker: MSSAR
No cookie policy popup found.
Error extracting table data for ticker MSSAR: Mess

Extracted row for TOPP: Toppoint Holdings Inc., 3, 01/23/2025, 01/21/2025, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318859525&type=DOC&symbol=TOPP&cdn=9c52d342b54d8d942ffa4bfd1a880b0d&companyName=Toppoint+Holdings+Inc.&formType=3&formDescription=Initial+statement+of+beneficial+ownership+of+securities&dateFiled=2025-01-23
Extracted row for TOPP: Toppoint Holdings Inc., 3, 01/23/2025, 01/21/2025, https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318859492&type=DOC&symbol=TOPP&cdn=0e3c00198656ca0b55b64b8a1c17aa55&companyName=Toppoint+Holdings+Inc.&formType=3&formDescription=Initial+statement+of+beneficial+ownership+of+securities&dateFiled=2025-01-23
Extracted row for TOPP: Toppoint Holdings Inc., 424B4, 01/22/2025, , https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=318856132&type=DOC&symbol=TOPP&cdn=7961f7011aee710a71fc097eb20381e7&companyName=Toppoint+Holdings+Inc.&formType=424B4&formDescription=Prospectus+%5BRule+424%28b